In [21]:
## estimate word embeddings from newspaper data
## code adapted from https://github.com/damian0604/embeddingworkshop/blob/main/04exercise.ipynb
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
import string
import re
import os
import pandas as pd
import csv


# tqdm allows you to display progress bars in loops
from tqdm import tqdm
from datetime import datetime

import gensim

# lets get more output
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [22]:
# get full set of news articles
!rm sample_data -r
!mkdir newspapers
!wget -O newspapers/articles.zip https://www.dropbox.com/sh/r6k4qk9flgz0agu/AAA5ZLsuOwk9UWiEsLAOFmDSa?dl=0
!unzip newspapers/articles.zip -d newspapers
!rm newspapers/articles.zip

rm: cannot remove 'sample_data': No such file or directory
mkdir: cannot create directory ‘newspapers’: File exists
--2021-07-21 12:55:36--  https://www.dropbox.com/sh/r6k4qk9flgz0agu/AAA5ZLsuOwk9UWiEsLAOFmDSa?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.82.18, 2620:100:6031:18::a27d:5112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.82.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /sh/raw/r6k4qk9flgz0agu/AAA5ZLsuOwk9UWiEsLAOFmDSa [following]
--2021-07-21 12:55:37--  https://www.dropbox.com/sh/raw/r6k4qk9flgz0agu/AAA5ZLsuOwk9UWiEsLAOFmDSa
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucac7fba53c8f7d43bcdc7d75101.dl.dropboxusercontent.com/zip_download_get/A2HJQS3MTgvZooc9K2B4WR5YpH7cOfmhThPyuPKRdIdSjkxNYs-aL6NwtuoM2ZWqaGAQIEk6Is5hcaEnB1AmfGfp_grsUqFD7qIApyK0x2L00Q# [following]
--2021-07-21 12:55:38--  https://ucac7fba53c8f7d43bcdc7d75101.dl.d

In [23]:
# load all texts
if 'artcls' in locals():
  del(artcls)

for filename in tqdm(os.listdir('newspapers')):
  if 'artcls' in locals():
    print(f'\nLoaded {artcls.shape[0]} articles')
    artcls = artcls.append(pd.read_csv('newspapers/'+filename))
  else:
    artcls = pd.read_csv('newspapers/'+filename)
print(f'Loaded {artcls.shape[0]} articles, done.')

  9%|▉         | 1/11 [00:15<02:35, 15.55s/it]


Loaded 263266 articles


 18%|█▊        | 2/11 [00:29<02:15, 15.05s/it]


Loaded 575291 articles


 27%|██▋       | 3/11 [00:45<02:02, 15.25s/it]

...


Loaded 1545861 articles


 91%|█████████ | 10/11 [01:30<00:05,  5.59s/it]


Loaded 1634513 articles


100%|██████████| 11/11 [02:00<00:00, 10.98s/it]

Loaded 2474182 articles, done.


In [24]:
artcls = artcls.reset_index()
artcls.text[0]

'was fehlt ... ... der Vorsatz    Der Vorsatz hat keinen guten Klang: Mit Vorsatz gehandelt zu haben, wird einem meist im Gericht vorgeworfen. Auch die guten Vorsätze haben immer den unangenehmen Beigeschmack von Schuld und Sühne - nicht zuletzt, weil sie zu 90 Prozent gebrochen werden. Woher der Brauch kommt, sich im neuen Jahr eine Änderung des Verhaltens vorzunehmen, ist unklar.    Am wahrscheinlichsten ist ein christlicher Ursprung, wie bei vielen Festtagsbräuchen - immerhin stammt das Wort Silvester vom Namenstag des Papstes Silvester (lateinisch für "Waldmensch"), der am 31. Dezember 335 starb. Möglicherweise sind die guten Vorsätze also eine katholische Erfindung: Die Sünden werden vergeben, aber nur, wenn man Besserung gelobt.    Die Wortherkunft der guten Vorsätze ist leichter zu bestimmen: Die Wurzel des Guten liegt im germanischen "goda" (passend, geeignet), das sich im 8. Jahrhundert zu "guot" (Besitz, Vermögen) weiterentwickelte. Vorsätze hießen im Mittelhochdeutschen "vür

In [25]:
artcls.shape

(2474182, 11)

In [1]:
# check if string
stringvar = [str == type(i) for i in artcls.text]
artcls = artcls[stringvar]

# cut into sentences
trans = str.maketrans('', '', string.punctuation) # translation scheme for removing punctuation
uniquesentences = set()
for review in tqdm(artcls.text):
    for sentence in sent_tokenize(review):
        # remove HTML tags in there
        sentence = re.sub(r"<.*?>"," ",sentence)
        sentence = sentence.translate(trans) 
        if sentence not in uniquesentences:
            uniquesentences.add(sentence.lower())

print(f"We now have {len(uniquesentences)} unique sentences.")

NameError: ignored

In [ ]:
del(artcls)

In [ ]:
with open('drive/MyDrive/uniquesentences.txt', mode='w') as fo:
  for sentence in uniquesentences:
    fo.write(sentence)

del(uniquesentences)

In [ ]:
tokenizedsentences = []

with open('drive/MyDrive/uniquesentences.txt', mode='r') as fi:
  reader = csv.reader(fi)
  for sentence in reader:
    tokenizedsentences.append(sentence.split())

In [ ]:
print(f"Started setting up the model at {datetime.now()}")
model = gensim.models.Word2Vec(size=300, min_count=100) # we want 300 dimensions and not overdo it with the features
model.build_vocab(tokenizedsentences)
print(f"Started training at {datetime.now()}")
model.train(tokenizedsentences, total_examples=model.corpus_count,  epochs=5)
# our model gets better if we use more epochs, but we can only do so if we use a list instead of a generator as input
# after all, you can only pass over a generator once.
# model.train(tokenizedsentences2, total_examples=model.corpus_count,  epochs=model.epochs)
print(f"Finished training at {datetime.now()}")

In [ ]:
model.save("np_emb")